In [5]:
import pandas as pd
import numpy as np


In [6]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity1")

#ws = Workspace(subscription_id, resource_group, workspace_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "udacitycluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
from azureml.core.conda_dependencies import CondaDependencies

# Specify parameter sampler
ps = RandomParameterSampling( {  
    "--C": choice(0.001, 0.01, 0.1, 1, 10, 20, 50, 100, 200, 500, 1000),
    "--max_iter": choice(100, 200, 300)
    }
)

# Specify a Policy: check the job every 2 iterations and if the primary metric falls outside the top 50% range, 
# Azure ML should terminate the job
policy = BanditPolicy(evaluation_interval=5, slack_factor=0.1)

#packages = CondaDependencies.create(
#    pip_packages=['azureml-defaults','scikit-learn']
#)

if "training" not in os.listdir():
    os.mkdir("./training")

# https://ml.azure.com/fileexplorerAzNB?wsid=/subscriptions/a022d83d-6229-4b5e-b039-b680692436b5/resourceGroups/azureai-test-east/providers/Microsoft.MachineLearningServices/workspaces/project-azureai-patton-test-east&tid=41b3409a-40cf-45e8-8201-ddfa1984ab3d&activeFilePath=Users/aspatton01/conda_dependencies.yml
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')
#sklearn_env = Environment("sklearn-env")
#sklearn_env.python.conda_dependencies = packages

src = ScriptRunConfig(source_directory="./",
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=8,
                                     max_concurrent_runs=4)


In [9]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

# Show run details
# RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")

RunId: HD_354605b0-1961-4b5b-b876-3a7c79c9cd12
Web View: https://ml.azure.com/runs/HD_354605b0-1961-4b5b-b876-3a7c79c9cd12?wsid=/subscriptions/a022d83d-6229-4b5e-b039-b680692436b5/resourcegroups/udacityclass/workspaces/udacitytestworkspace&tid=41b3409a-40cf-45e8-8201-ddfa1984ab3d

Streaming azureml-logs/hyperdrive.txt

[2024-02-29T21:12:52.722131][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-02-29T21:12:53.2270325Z][SCHEDULER][INFO]Scheduling job, id='HD_354605b0-1961-4b5b-b876-3a7c79c9cd12_0' 
[2024-02-29T21:12:53.3423010Z][SCHEDULER][INFO]Scheduling job, id='HD_354605b0-1961-4b5b-b876-3a7c79c9cd12_1' 
[2024-02-29T21:12:53.4886544Z][SCHEDULER][INFO]Scheduling job, id='HD_354605b0-1961-4b5b-b876-3a7c79c9cd12_2' 
[2024-02-29T21:12:53.5884939Z][SCHEDULER][INFO]Scheduling job, id='HD_354605b0-1961-4b5b-b876-3a7c79c9cd12_3' 
[2024-02-29T21:12:53.532852][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2024

In [11]:
import joblib
# Get your best run and save the model from that run.

hd_best_run = hyperdrive_run.get_best_run_by_primary_metric()
hd_best_model = hd_best_run.register_model(model_name="hd_best_model" , model_path="./")

best_run_metrics = hd_best_run.get_metrics()
parameter_values = hd_best_run.get_details()
best_run_filenames = hd_best_run.get_file_names()


print("Best Run Id: ", hd_best_run.id)
print("Best run metrics: ", best_run_metrics)
print("Accuracy: ", best_run_metrics["Accuracy"])
print("Best run details ", parameter_values)
print("Parameters: ", parameter_values["runDefinition"]["arguments"])

Best Run Id:  HD_354605b0-1961-4b5b-b876-3a7c79c9cd12_4
Best run metrics:  {'Regularization Strength:': 1000.0, 'Max iterations:': 100, 'Accuracy': 0.9062879339645545}
Accuracy:  0.9062879339645545
Best run details  {'runId': 'HD_354605b0-1961-4b5b-b876-3a7c79c9cd12_4', 'target': 'udacitycluster', 'status': 'Completed', 'startTimeUtc': '2024-02-29T21:25:46.108452Z', 'endTimeUtc': '2024-02-29T21:26:09.152629Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', '_azureml.ClusterName': 'udacitycluster', 'ContentSnapshotId': '6050f9d0-05bb-4b06-8488-21c913ab3753', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '1000', '--max_iter', '100'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'udacitycluster', 'dataReference

In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

csv_url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'    

# Create a TabularDataset using TabularDatasetFactory
ds = TabularDatasetFactory.from_delimited_files(path=csv_url)

In [13]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [14]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target = cpu_cluster,
    experiment_timeout_minutes=15,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    enable_onnx_compatible_models=True,
    n_cross_validations=2)

In [15]:
# Submit your automl run

remote_run = exp.submit(automl_config, show_output=False)
remote_run.wait_for_completion()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity1,AutoML_2e4bada9-b5a6-437e-a9a0-516e6a526204,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


{'runId': 'AutoML_2e4bada9-b5a6-437e-a9a0-516e6a526204',
 'target': 'udacitycluster',
 'status': 'Completed',
 'startTimeUtc': '2024-02-29T21:30:47.501761Z',
 'endTimeUtc': '2024-02-29T21:55:06.027988Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 15 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'udacitycluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity1","subscription_id":"a022d83d-6229-4b5e-b039-b680692436b5","resource_group":"udacityclass","workspace_name":"udacitytestworkspace","region":"eastus2","compute_target":"udacitycluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run"

In [16]:
# Retrieve and save your best automl model.

best_run, fitted_model = remote_run.get_output()

print(best_run)
print(fitted_model)

# get_metrics()
# Returns the metrics
print("Best run metrics :",best_run.get_metrics())
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

Run(Experiment: udacity1,
Id: AutoML_2e4bada9-b5a6-437e-a9a0-516e6a526204_21,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('18', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.7, eta=0.1, gamma=0.1, max_depth=9, max_leaves=511, n_estimators=25, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu

In [17]:
cpu_cluster.delete()


In [ ]:
# Retrieve and save your best automl model.

best_run, fitted_model = remote_run.get_output()

print(best_run)
print(fitted_model)

print("Best run metrics: ", best_run.get_metrics())
print("Best run details: ", best_run.get_details())